# RNN : LSTM


In [1]:
# import
import pickle 

import numpy as np

from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Input
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
# from keras.wrappers.scikit_learn import KerasRegressor
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, mean_absolute_error

# reproductibility
np.random.seed(42)

In [2]:
## ICI ON RÈGLE QUELLE TAILLE DE FENETRE ON VEUT UTILISER (POUR NE PAS DUPLIQUER LE CODE)
taille_fenetre_to_run = "courte"
assert taille_fenetre_to_run in ["courte", "moyenne", "longue"]

In [3]:
if taille_fenetre_to_run == "courte":
    data = pickle.load(open("Data/donnees_courte.pkl", "rb"))
elif taille_fenetre_to_run == "moyenne":
    data = pickle.load(open("Data/donnees_moyenne.pkl", "rb"))
else:
    data = pickle.load(open("Data/donnees_longue.pkl", "rb"))

In [13]:
features_binary = pickle.load(open("Data/features_binary.pkl", "rb"))
features_binary 

{0: 'cycle_life',
 1: 'C1',
 2: 'C2',
 3: 'Q1',
 4: 'IR',
 5: 'QC',
 6: 'Tavg',
 7: 'Tmin',
 8: 'Tmax',
 9: 'chargetime',
 10: 'pulse_width',
 11: 'cycle_to_x%',
 12: 'cutoff_currents',
 13: 'charging_rest',
 14: 'after_discharging_rest',
 15: 'before_discharging_rest',
 16: 'IR_rest',
 17: 'barcode_0',
 18: 'barcode_1',
 19: 'barcode_2',
 20: 'barcode_3',
 21: 'barcode_4',
 22: 'barcode_5',
 23: 'barcode_6',
 24: 'cycle_normalized'}

In [4]:
data.keys()

dict_keys(['X_np_label', 'X_np_binary', 'y_np', 'X_df_label', 'X_df_binary', 'y_df'])

In [5]:
X_np_binary = data["X_np_binary"]
y_np = data["y_np"]

In [6]:
# Vérifier que toutes les données de X_np_binary sont entre 0 et 1
for i in range(X_np_binary.shape[0]):
    for j in range(X_np_binary.shape[1]):
        for k in range(X_np_binary.shape[2]):
            assert X_np_binary[i, j, k] >= 0 and X_np_binary[i, j, k] <= 1

In [7]:
def create_lstm(units=100, dropout_rate=0.2, learning_rate=0.001, input_shape = (X_np_binary.shape[1], X_np_binary.shape[2]), activation='tanh'):
    # pour ajouter des couches 
    model = Sequential()

    # shape de l'input (sibnon warning plus tar)
    model.add(Input(shape=input_shape))

    # units : 100, Plus ce nombre est élevé, plus le modèle peut capturer de relations complexes dans les données, mais cela augmente aussi le coût computationnel.
    # activation : tanh, fonction d'activation tanh (classique dans les LSTM)
    model.add(LSTM(units, activation=activation))

    # éviter surapprentissage
    model.add(Dropout(dropout_rate)) 

    # output pour un problème de régresssion 
    model.add(Dense(1)) 

    # optimizer adam 
    # loss : mse : typique pour un problème de régression
    # metrics : mae : pour voir l'erreur moyenne
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse') # , metrics=['mae'])) # mse pour un problème de régression ?

    return model

In [8]:
keras_regressor = KerasRegressor(
    loss='mse', # peut etre autre à tester ? 
    model=create_lstm, 
    input_shape=(X_np_binary.shape[1], X_np_binary.shape[2]), 
    verbose=1 
)

In [9]:
param_grid = {
    'model__units': [50, 100], #, 200],
    'model__dropout_rate': [0.2, 0.3], # , 0.5],
    'model__activation': ['tanh', 'relu'],
    'model__learning_rate': [0.001, 0.01], #, 0.1],
    'epochs': [100], #, 200],
    'batch_size': [32] #, 64] #, 128]
}

In [10]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [11]:
grid = GridSearchCV(estimator=keras_regressor, param_grid=param_grid, cv=KFold(n_splits=5, random_state=42, shuffle=True), scoring='neg_mean_squared_error', verbose=1, n_jobs=-1, refit=True)

grid_result = grid.fit(X_np_binary, y_np, callbacks=[early_stopping])

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

means = grid_result.cv_results_['mean_test_score']
print("Means: ", means)

best_model = grid_result.best_estimator_
print(best_model)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - loss: 8227.9580
Epoch 2/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - loss: 8281.6934
Epoch 2/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - loss: 8243.2627
Epoch 2/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - loss: 8355.1699
Epoch 2/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - loss: 8250.0869
Epoch 2/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - loss: 6246.3140
Epoch 3/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 6308.0806
Epoch 3/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 6245.0620
Epoch 3/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 5s 33ms/step - loss: 7442.0391
Epoch 2/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 5s 34ms/step - loss: 7623.4258
Epoch 2/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 5s 34ms/step - loss: 7724.4146
Epoch 2/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 6

/Users/zoemarquis/Documents/projet_industrie/projet_batteries/mon_venv/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/zoemarquis/Documents/projet_industrie/projet_batteries/mon_venv/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/zoemarquis/Documents/projet_industrie/projet_batteries/mon_venv/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 279, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/zoemarquis/Documents/projet_industrie/projet_batteries/mon_venv

 99/118 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 44.0161Epoch 1/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - loss: 44.067225
Epoch 87/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 7281.3481
Epoch 46/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 390269.3750
Epoch 81/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 3s 28ms/step - loss: 57.955317
Epoch 62/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 7281.49170
Epoch 47/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - loss: 31584.4531
Epoch 2/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 715566.8750
Epoch 82/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - loss: 65.267452
Epoch 55/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 7137.02542
Epoch 48/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 1782.10123
Epoch 3/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 267616.0625
Epoch 83/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - loss: 46.13912
Epoch 88/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 7141.9

/Users/zoemarquis/Documents/projet_industrie/projet_batteries/mon_venv/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/zoemarquis/Documents/projet_industrie/projet_batteries/mon_venv/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/zoemarquis/Documents/projet_industrie/projet_batteries/mon_venv/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 279, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/zoemarquis/Documents/projet_industrie/projet_batteries/mon_venv

 19/118 ━━━━━━━━━━━━━━━━━━━━ 3s 39ms/step - loss: 58.5802Epoch 1/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 4s 36ms/step - loss: 125.2768
Epoch 7/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 5s 39ms/step - loss: 61.9047
Epoch 75/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 432.4998
Epoch 54/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 4s 36ms/step - loss: 133.9736
Epoch 15/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - loss: 62.25144
Epoch 80/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 4s 36ms/step - loss: 55.89002
Epoch 76/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 4s 34ms/step - loss: 53.00654
Epoch 83/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - loss: 431.6026
Epoch 55/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 4s 34ms/step - loss: 107.9378
Epoch 8/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 4s 32ms/step - loss: 53.28627
Epoch 76/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 5s 31ms/step - loss: 6164.2183
Epoch 2/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: 435.1915
Epoch 56/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 5s 35ms/step - loss: 135.5079
 90/118 ━━━

...

118/118 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - loss: 4452.2109
Epoch 84/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - loss: nan6.502
Epoch 82/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: nan0.356
Epoch 89/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 4341.7363
Epoch 85/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: nan7.872
Epoch 83/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: nan4.352
Epoch 90/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 4205.0269
Epoch 86/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: nan8.324
Epoch 84/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: nan7.113
Epoch 91/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 4304.6758
Epoch 87/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - loss: nan
Epoch 85/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: nan3.917
Epoch 92/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - loss: 7442.0698
115/118 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: nanEpoch 88/100
118/118 ━━━

/Users/zoemarquis/Documents/projet_industrie/projet_batteries/mon_venv/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/zoemarquis/Documents/projet_industrie/projet_batteries/mon_venv/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/zoemarquis/Documents/projet_industrie/projet_batteries/mon_venv/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 279, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/zoemarquis/Documents/projet_industrie/projet_batteries/mon_venv

118/118 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - loss: nan6.447
Epoch 95/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - loss: 5254.8364
Epoch 97/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: nan7.954
Epoch 96/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 4823.8433
Epoch 98/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: nan5.017
Epoch 97/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 5008.0034
Epoch 99/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: nan0.971
Epoch 98/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 4745.5688
Epoch 100/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: nan7.224
Epoch 99/100
118/118 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 4701.2832
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: nan
Epoch 100/100
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/stepep - loss: na
118/118 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: nan
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
Epoch 1/100


/Users/zoemarquis/Documents/projet_industrie/projet_batteries/mon_venv/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:982: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/zoemarquis/Documents/projet_industrie/projet_batteries/mon_venv/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 971, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/zoemarquis/Documents/projet_industrie/projet_batteries/mon_venv/lib/python3.11/site-packages/sklearn/metrics/_scorer.py", line 279, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/zoemarquis/Documents/projet_industrie/projet_batteries/mon_venv

148/148 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 7383.3149
Epoch 2/100
148/148 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 4360.5049
Epoch 3/100
148/148 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 2778.8657
Epoch 4/100
148/148 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 1747.0392
Epoch 5/100
148/148 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 1031.8528
Epoch 6/100
148/148 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 587.0025
Epoch 7/100
148/148 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 327.0373
Epoch 8/100
148/148 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 179.6079
Epoch 9/100
148/148 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 109.0512
Epoch 10/100
148/148 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - loss: 74.4408
Epoch 11/100
148/148 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 57.8309
Epoch 12/100
148/148 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 54.2743
Epoch 13/100
148/148 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 50.9769
Epoch 14/100
148/148 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 48.9280
Epoch 15/100


In [ ]:
# Best: -0.709300 using {'batch_size': 32, 'epochs': 100, 'model__activation': 'tanh', 'model__dropout_rate': 0.2, 'model__learning_rate': 0.001, 'model__units': 100}
# Means:  [-1.28047135e+00 -7.09300163e-01 -1.50861980e+00 -2.91220027e+00
#  -3.32321884e+00 -1.73152464e+00 -2.25481452e+00 -4.05518777e+00
#              nan -1.23089762e+03             nan             nan
#  -4.33338616e+03 -1.99504134e+05             nan             nan]
# KerasRegressor(
# 	model=<function create_lstm at 0x168294680>
# 	build_fn=None
# 	warm_start=False
# 	random_state=None
# 	optimizer=rmsprop
# 	loss=mse
# 	metrics=None
# 	batch_size=32
# 	validation_batch_size=None
# 	verbose=1
# 	callbacks=None
# 	validation_split=0.0
# 	shuffle=True
# 	run_eagerly=False
# 	epochs=100
# 	input_shape=(50, 25)
# 	model__activation=tanh
# 	model__dropout_rate=0.2
# 	model__learning_rate=0.001
# 	model__units=100
# )

In [18]:
# save best model
pickle.dump(best_model, open("Data/best_model.pkl", "wb"))

In [19]:
print(type(best_model))

<class 'scikeras.wrappers.KerasRegressor'>


In [20]:
print(dir(best_model))

['X_dtype_', 'X_shape_', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__name__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__sklearn_clone__', '__slotnames__', '__str__', '__subclasshook__', '__weakref__', '_build_keras_model', '_build_request_for_signature', '_check_feature_names', '_check_model_compatibility', '_check_model_param', '_check_n_features', '_compile_kwargs', '_doc_link_module', '_doc_link_template', '_doc_link_url_param_generator', '_ensure_compiled_model', '_estimator_type', '_fit', '_fit_callbacks', '_fit_keras_model', '_fit_kwargs', '_get_compile_kwargs', '_get_default_requests', '_get_doc_link', '_get_metadata', '_get_metadata_request', '_get_param_names', '_get_tags', '_initialize', '_initialize_callbacks', '_more_tags', '_pr

In [21]:
best_model.model_

<Sequential name=sequential, built=True>

In [22]:
best_model.model_.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 100)            │        50,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 151,505 (591.82 KB)

 Trainable params: 50,501 (197.27 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 101,004 (394.55 KB)

In [26]:
best_model.model_.save(f"Models/lstm_{taille_fenetre_to_run}_gridsearch.h5")

In [27]:
# MSE, MAE, RMSE, MAPE
y_pred = best_model.predict(X_np_binary)
mse = mean_squared_error(y_np, y_pred)
mae = mean_absolute_error(y_np, y_pred)
mape = mean_absolute_percentage_error(y_np, y_pred)
rmse = np.sqrt(mse)

print(f"MSE: {mse}")
print(f"MAE: {mae}")
print(f"MAPE: {mape}")
print(f"RMSE: {rmse}")

data_metrics = {
    "mse": mse,
    "mae": mae,
    "mape": mape,
    "rmse": rmse
}
# save metrics
pickle.dump(data_metrics, open(f"Data/metrics_lstm_{taille_fenetre_to_run}_gridsearch.pkl", "wb"))

148/148 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
MSE: 1.3113283775260178
MAE: 0.8745182968257111
MAPE: 0.009386523798784624
RMSE: 1.1451324716058042


In [30]:
# import matplotlib.pyplot as plt

# # Récupérer l'historique du meilleur modèle
# history = grid_result.best_estimator_.model_.history

# # Tracer la courbe de la perte
# plt.figure(figsize=(12, 6))

# # Loss curve
# plt.subplot(1, 2, 1)
# plt.plot(history.history['loss'], label='Loss')
# plt.plot(history.history['val_loss'], label='Validation Loss')
# plt.title('Loss over epochs')
# plt.xlabel('Epochs')
# plt.ylabel('Loss')
# plt.legend()

# # Optionnel : Tracer l'accuracy si disponible
# if 'accuracy' in history.history:
#     plt.subplot(1, 2, 2)
#     plt.plot(history.history['accuracy'], label='Accuracy')
#     plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
#     plt.title('Accuracy over epochs')
#     plt.xlabel('Epochs')
#     plt.ylabel('Accuracy')
#     plt.legend()

# # Afficher les courbes
# plt.tight_layout()
# plt.show()

In [ ]:
! ne pas re run le code au dessus -> 
# extraire les parametres du meilleur modele
# ou juste charger le best_model 
# best_model.model_.save(f"Models/lstm_{taille_fenetre_to_run}_gridsearch.h5")

from tensorflow.keras.models import load_model

# Charger le modèle sauvegardé
loaded_model = load_model(f"Models/lstm_{taille_fenetre_to_run}_gridsearch.h5")

## Explicabilité

In [32]:
import lime
from lime.lime_tabular import LimeTabularExplainer

explainer = LimeTabularExplainer(
    training_data=X_np_binary, 
    mode="regression", 
    training_labels=y_np,
    feature_names=features_binary,
    class_names=["SOH"]
)

explanation = explainer.explain_instance(X_np_binary[0], best_model.predict)
explanation.show_in_notebook()

IndexError: boolean index did not match indexed array along axis 1; size of axis is 50 but size of corresponding boolean axis is 25

In [12]:


# best_model.save("Models/model_{taille_fenetre_to_run}_lstm_gridsearch.h5")